In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Установка зависимотсей

In [4]:
!pip install transformers accelerate
!huggingface-cli login
!pip install --upgrade pip
!pip install --force-reinstall "unsloth[cu121-torch240] @ git+https://github.com/unslothai/unsloth.git"
!pip install --force-reinstall unsloth_zoo

In [1]:
!pip install PyPDF2
!pip install langchain
!pip install huggingface_hub
!pip install onnxruntime
!pip install transformers
!pip install sentence-transformers

In [ ]:
!pip install --upgrade numpy


In [2]:
!pip install faiss-cpu

In [3]:
!pip install protobuf==3.20.3

### RAG

In [3]:
import os
import csv
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from huggingface_hub import hf_hub_download
import onnxruntime as ort
from transformers import AutoTokenizer
import numpy as np
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

#### RAG creating

In [ ]:
path_to_pdf = "/content/drive/MyDrive/Vk_hack/data/pdf"
path_to_csv = "//content/drive/MyDrive/Vk_hack/data/csv"

def read_pdf(pdf_folder):
    docs = []
    for file in os.listdir(pdf_folder):
        if file.endswith(".pdf"):
            path = os.path.join(pdf_folder, file)
            reader = PdfReader(path)
            for i, page in enumerate(reader.pages):
                text = page.extract_text()
                if text:
                    docs.append(
                        Document(
                            page_content=text,
                            metadata={"source": file, "page": i + 1}
                        )
                    )
    return docs


def read_csv(csv_folder):
    docs = []
    for file in os.listdir(csv_folder):
        if file.endswith(".csv"):
            path = os.path.join(csv_folder, file)
            with open(path, "r", encoding="utf-8", errors="ignore") as f:
                reader = csv.reader(f)
                header = next(reader, None)  # первая строка (заголовки)
                for i, row in enumerate(reader):
                    # собираем строку вида "col1: val1, col2: val2, ..."
                    if header:
                        text = ", ".join([f"{col}: {val}" for col, val in zip(header, row)])
                    else:
                        text = ", ".join(row)
                    docs.append(
                        Document(
                            page_content=text,
                            metadata={"source": file, "row": i + 1}
                        )
                    )
    return docs


# Load all documents
all_docs = read_pdf(path_to_pdf) + read_csv(path_to_csv)
print(f"Загружено документов: {len(all_docs)}")

# Split on chunks with saving the metadata
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunked_docs = splitter.split_documents(all_docs)

Загружено документов: 336508


In [ ]:
def generate_embeddings(docs, model_name="sentence-transformers/all-MiniLM-L6-v2", batch_size=256):
    model = SentenceTransformer(model_name)

    texts = [doc.page_content for doc in docs]
    metas = [doc.metadata for doc in docs]

    for start in range(0, len(texts), batch_size):
        batch_texts = texts[start:start+batch_size]
        batch_metas = metas[start:start+batch_size]

        batch_embeddings = model.encode(
            batch_texts,
            batch_size=batch_size,
            convert_to_numpy=True,
            show_progress_bar=False
        )

        # Возвращаем пачку как генератор
        for i, emb in enumerate(batch_embeddings):
            yield {
                "embedding": emb,
                "content": batch_texts[i],
                "metadata": batch_metas[i]
            }

In [ ]:
# Создаём FAISS-индекс
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
d = model.get_sentence_embedding_dimension()
index = faiss.IndexFlatL2(d)

texts, metas = [], []

# Генеративная обработка
for i, item in enumerate(generate_embeddings(chunked_docs, batch_size=256)):
    index.add(np.array([item["embedding"]], dtype="float32"))
    texts.append(item["content"])
    metas.append(item["metadata"])

else:
    print(f"Добавлено {i} эмбеддингов в индекс")

print("Размер индекса:", index.ntotal)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Добавлено 527981 эмбеддингов в индекс
Размер индекса: 527982


#### Saving Index

In [ ]:
import pickle

# Сохраняем индекс
faiss.write_index(index, "/content/drive/MyDrive/Vk_hack/faiss_index.idx")

# Сохраняем тексты и метаданные
with open("texts_metas.pkl", "wb") as f:
    pickle.dump({"texts": texts, "metas": metas}, f)

#### Loading Index

In [ ]:
import pickle

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Загружаем индекс
index = faiss.read_index("/content/drive/MyDrive/Vk_hack/faiss_index.idx")

# Загружаем тексты и метаданные
with open("/content/drive/MyDrive/Vk_hack/texts_metas.pkl", "rb") as f:
    data = pickle.load(f)
texts = data["texts"]
metas = data["metas"]

#### Search for relevant documents

In [ ]:
# Поиск релевантых документов
def search(query, model, index, texts, metas, top_k=5):
    query_emb = model.encode([query], convert_to_numpy=True).astype("float32")
    distances, indices = index.search(query_emb, top_k)
    return [
        {"text": texts[i], "metadata": metas[i], "distance": float(distances[0][j])}
        for j, i in enumerate(indices[0])
    ]


### Promt compilation and generation function

In [16]:
def generate_optimized_sql_few_shot(query, retrieved_docs, model, tokenizer):
    context_text = """
"""
    if retrieved_docs:
        context_text += "\n\nAdditional context:\n" + "\n".join([doc['text'] for doc in retrieved_docs])


    few_shot_examples = """
    Example 1:
    Original SQL:
    SELECT * FROM orders WHERE user_id IN (SELECT id FROM users WHERE status='active');
    Optimized SQL:
    SELECT o.*
    FROM orders o
    JOIN users u ON o.user_id = u.id
    WHERE u.status='active';

    Example 2:
    Original SQL:
    SELECT u.id, (SELECT COUNT(*) FROM orders o WHERE o.user_id=u.id) AS order_count FROM users u;
    Optimized SQL:
    SELECT u.id, COUNT(o.id) AS order_count
    FROM users u
    LEFT JOIN orders o ON u.id = o.user_id
    GROUP BY u.id;

    Example 3:
    Original SQL:
    SELECT o.id, o.total, u.name
    FROM orders o
    JOIN users u ON o.user_id = u.id
    WHERE u.status='active' AND o.total > 100;
    Optimized SQL:
    SELECT o.id, o.total, u.name
    FROM orders o
    JOIN users u ON o.user_id = u.id
    WHERE u.status='active' AND o.total > 100;
    -- Этот запрос уже оптимален, изменений не требуется

    Example 4:
    Original SQL:
    WITH recent_orders AS (
        SELECT * FROM orders WHERE created_at > NOW() - INTERVAL '30 days'
    )
    SELECT u.id, u.name, COUNT(r.id)
    FROM users u
    JOIN recent_orders r ON u.id = r.user_id
    GROUP BY u.id, u.name;
    Optimized SQL:
    SELECT u.id, u.name, COUNT(o.id)
    FROM users u
    JOIN orders o ON u.id = o.user_id
    WHERE o.created_at > NOW() - INTERVAL '30 days'
    GROUP BY u.id, u.name;

    Example 5:
    Original SQL:
    SELECT u.id, u.name,
          (SELECT MAX(p.created_at) FROM payments p WHERE p.user_id=u.id) AS last_payment
    FROM users u;
    Optimized SQL:
    SELECT u.id, u.name, MAX(p.created_at) AS last_payment
    FROM users u
    LEFT JOIN payments p ON u.id = p.user_id
    GROUP BY u.id, u.name;

    Example 6:
    Original SQL:
    WITH recent_orders AS (
        SELECT * FROM orders WHERE created_at > NOW() - INTERVAL '30 days'
    ),
    completed_payments AS (
        SELECT * FROM payments WHERE status = 'completed'
    )
    SELECT u.id, u.name, r.id AS order_id, r.total, p.id AS payment_id, p.created_at AS payment_date
    FROM users u
    JOIN recent_orders r ON u.id = r.user_id
    LEFT JOIN completed_payments p ON r.id = p.order_id
    WHERE u.status = 'active'
    ORDER BY r.total DESC;
    Optimized SQL:
    SELECT u.id, u.name, o.id AS order_id, o.total, p.id AS payment_id, p.created_at AS payment_date
    FROM users u
    JOIN orders o
        ON u.id = o.user_id
        AND o.created_at > NOW() - INTERVAL '30 days'
    LEFT JOIN payments p
        ON o.id = p.order_id
        AND p.status = 'completed'
    WHERE u.status = 'active'
    ORDER BY o.total DESC;

    Example 7:
    Original SQL:
    SELECT u.id, u.name, o.id AS order_id, o.total,
          SUM(p.amount) OVER(PARTITION BY u.id ORDER BY p.created_at) AS cumulative_payments
    FROM users u
    JOIN orders o ON u.id = o.user_id
    LEFT JOIN payments p ON o.id = p.order_id
    WHERE u.status='active';
    Optimized SQL:
    SELECT u.id, u.name, o.id AS order_id, o.total,
          SUM(p.amount) OVER(PARTITION BY u.id ORDER BY p.created_at) AS cumulative_payments
    FROM users u
    JOIN orders o ON u.id = o.user_id
    LEFT JOIN payments p ON o.id = p.order_id AND p.status='completed'
    WHERE u.status='active';
    -- Добавлен фильтр p.status='completed' для оптимизации
    """


    prompt = f"""
I am a senior SQL developer with many years of experience, deeply understanding the philosophy of writing SQL code, its subtle points, and potential vulnerabilities. Please optimize the following SQL query as efficiently and safely as possible. Full context of the database and environment is provided.

Tasks:
1. Analyze the query in terms of performance: estimate number of operations, iterations, and volume of data read.
2. Identify bottlenecks and potential vulnerabilities, including SQL injection risks, suboptimal joins, unnecessary subqueries, and aggregations.
3. Apply best practices for query optimization where possible, such as: proper indexing, reordering of filter conditions (WHERE, ON), optimizing join order, minimizing data volume in intermediate steps, using window functions or CTEs.
4. Provide a revised query that is fully detailed, safe, and includes an explanation of each change. Preserve the semantic meaning of the original query. If a JOIN can be safely removed without changing the semantics, do so.
5. Take into account the specific database system, version, and configuration. For repeated computations, suggest caching or materialization strategies.
6. Emphasize security: eliminate any risks of data leakage or unintended operations.
7. Additionally, create a concise operations map with estimated time and computational complexity. Where possible, provide alternative optimization options and tools for monitoring and diagnostics.

For analysis, I provide the original SQL query, the execution plan, and database structure (if necessary).

Context:
{context_text}

{few_shot_examples}

Now optimize this SQL query:

Original SQL:
{query}


Optimized SQL:
"""

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=150,         # Максимальное количество генерируемых токенов
        do_sample=True,             # Включаем сэмплинг (вместо жадного выбора)
        temperature=0.7,            # "Температура" для сэмплинга
        top_k=30,                   # Ограничиваем выбор 50 самыми вероятными токенами
        top_p=0.9,                  # Нуклеарный сэмплинг с порогом 0.9
        num_beams=2,                # Количество лучей в beam search
        repetition_penalty=2.5,     # Штраф за повторения токенов
        early_stopping=True,        # Останавливать генерацию при EOS токене
        num_return_sequences=1      # Количество возвращаемых вариантов генерации
    )

    result = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    pretty_output = "\nOptimized SQL and explanation:\n" + "="*80 + "\n\n"
    pretty_output += result + "\n\n" + "="*80 + "\n"

    return pretty_output


### Inference

In [4]:
import torch
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()


In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

merged_model_path = "/content/drive/MyDrive/Vk_hack/sqlcoder-finetuned_3"


tokenizer = AutoTokenizer.from_pretrained(merged_model_path)


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

# Загружаем уже слитую модель
model_sql = AutoModelForCausalLM.from_pretrained(
    merged_model_path,
    device_map="auto",
    quantization_config=bnb_config
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [4]:
query = """
WITH recent_orders AS (
    SELECT *
    FROM orders
    WHERE created_at > NOW() - INTERVAL '30 days'
),
completed_payments AS (
    SELECT *
    FROM payments
    WHERE status = 'completed'
)
SELECT u.id,
       u.name,
       r.id AS order_id,
       r.total,
       p.id AS payment_id,
       p.created_at AS payment_date
FROM users u
JOIN recent_orders r ON u.id = r.user_id
LEFT JOIN completed_payments p ON r.id = p.order_id
WHERE u.status = 'active'
ORDER BY r.total DESC;
"""
#retrieved = search(query, model, index, texts, metas, top_k=5)
answer = generate_optimized_sql_few_shot(query, {}, model_sql, tokenizer)

print("Ответ:", answer)